In [15]:
import os
from pydub import AudioSegment
from math import ceil
from groq import Groq

GROQ_API_KEY = "API_KEY_HERE"

MP3_FILE_PATH = '../data/DS DATA101- Introduction to Data Industry Ders 4.mp3'
OUTPUT_FOLDER = '../data/trimmed_audio'

In [16]:
def trim_mp3(input_file, output_folder, max_size_mb=20, min_size_mb=10, overlap_seconds=10):
    # Load the audio file
    audio = AudioSegment.from_mp3(input_file)
    
    # Calculate file size in bytes
    file_size = os.path.getsize(input_file)
    
    # Calculate number of parts
    num_parts = ceil(file_size / (max_size_mb * 1024 * 1024))
    
    # Calculate part duration in milliseconds
    part_duration = len(audio) // num_parts
    
    # Ensure part size is within constraints
    while (part_duration * file_size / len(audio)) / (1024 * 1024) < min_size_mb:
        num_parts -= 1
        part_duration = len(audio) // num_parts
    
    # Convert overlap to milliseconds
    overlap_ms = overlap_seconds * 1000
    
    # Trim audio into parts
    parts = []
    for i in range(num_parts):
        start = max(0, i * part_duration - overlap_ms)
        end = min(len(audio), (i + 1) * part_duration + overlap_ms)
        part = audio[start:end]
        parts.append(part)
    
    # Export parts
    for i, part in enumerate(parts):
        output_file = output_folder + f"/output_part_{i+1}.mp3"
        part.export(output_file, format="mp3")
        print(f"Exported {output_file}")


In [17]:
trim_mp3(MP3_FILE_PATH, OUTPUT_FOLDER)

Exported ../data/trimmed_audio/output_part_1.mp3
Exported ../data/trimmed_audio/output_part_2.mp3
Exported ../data/trimmed_audio/output_part_3.mp3


In [6]:
client = Groq(api_key=GROQ_API_KEY)
model = 'whisper-large-v3'

def audio_to_text(filepath):
    """
    Transcribes the audio file located at the given filepath into text.

    Parameters:
    filepath (str): The path to the audio file.

    Returns:
    str: The transcribed text from the audio file.
    """
    with open(filepath, "rb") as file:
        transcription = client.audio.transcriptions.create(
            file=(filepath, file.read()),
            model="whisper-large-v3",
        )
    return transcription.text

filepath = OUTPUT_FOLDER + "/output_part_1.mp3"
transcription_text = audio_to_text(filepath)
print(transcription_text[:50])

 En son hatırlıyorsanız arkadaşlar veri bilimi iş 


In [13]:
from langchain import PromptTemplate

template = """
You are an Educational Transcribed Content Editor specializing in Turkish Data Science lectures. Your task is to edit the provided transcription while adhering to the following guidelines:

1. Maintain the conversational style and flow of the original lecture.
2. Correct any typos, grammatical errors, or mistranslated words.
3. Ensure technical terms related to Data Science are accurately represented.
4. Preserve the speaker's original intent and tone.
5. Make minimal changes necessary for clarity and correctness.
6. The final transcription must be in Turkish.

Original Transcription:
{transcription}

Please provide the edited version of the transcription below:

Edited Transcription:
"""

prompt = PromptTemplate(
    input_variables=["transcription"],
    template=template
)

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    api_key=GROQ_API_KEY
    )
pipeline = prompt | llm

In [14]:
edited_transcription = pipeline.invoke({"transcription": transcription_text})
print(edited_transcription.content)

Düzenlenmiş Transkript:
En son hatırlıyorsanız arkadaşlar, veri bilimi iş akışı veya uygulama çerçevesi olarak tabir ettiğim kısımdan bahsetmiştik. Temelimizi problemi anlamak oluşturuyordu aslında. Yani veriden önce üzerinde çalışacağımız problemi anlamamız gerekiyordu. Daha sonra veri toplama, veri hazırlama, keşifsel veri analizi, analitik, optimizasyon derken zaten tam bir döngüden bahsetmiştik. Buradan sonra biraz bu adımların detaylarına gireceğiz. Bazıları gerçekten çok detaylı gidecek. Özellikle o detaylı kısımlarda birçok yerde beni şey derken bulacaksınız. İşte bundan ilerideki derste bahsedeceğiz falan gibi. Ama olabildiğince de kısa kısa aklınızda verebilecek şekilde oluşmasına yardımcı olmaya çalışacağım oralarda. Buradaki amacımız size overall bütün resmi sunmak diyebilirim.

Şimdi buralara gelmeden önce biraz daha önce işlediğimiz şeylere bir bakalım istiyorum. Biraz vakit de geçti çünkü. Burada bazı olmak çalışmalardan bahsetmiştik. Burada hatta sizlerden de örnekler al